In [ ]:
import os
import re

In [ ]:
# Configurable values

_struct_name = "YourServiceHere"
_interface_name = "YourInterfaceHere"
_path1 = "/full/path/to/your/package"

In [ ]:
_str1 = f"func [\\S \\t]+\\b{_struct_name}[ \\t\\n]*\\)[ \\t]+([A-Za-z0-9_]+)([\\S \\t]+)"
_re1 = re.compile(_str1)

_re_whitespace = re.compile(r"[\t ]+")

_re_comma_ending_parentheses = re.compile(r",[\t ]*\)")
_re_opening_parentheses_whitespace = re.compile(r"\([ \t]+")

def _process_file(file_path: str) -> list[str]:
    with open(file_path, "r", encoding="utf-8") as infile:
        data = infile.read()
    public_functions: list[str] = []
   
    # resettable values
    trying_to_find = False
    bracket_found = False
    current_function_name: list[str] = []

    # start processing
    for line in data.split("\n"):
        if trying_to_find:
            current_line = []
            line = _re_whitespace.sub(" ", line)
            for c1 in line:
                if c1 == '{':
                    bracket_found = True
                    break
                current_line.append(c1)
            function_name = "".join(current_line).rstrip()
            current_function_name.append(function_name)
        else:
            m = _re1.search(line)
            if m:
                function_name = m.group(1)
                if function_name[0].isupper():
                    trying_to_find = True
                    current_function_name.append(function_name)
                    remaining = m.group(2)
                    for c1 in remaining:
                        if c1 == '{':
                            bracket_found = True
                            break
                        current_function_name.append(c1)
        if bracket_found:
            function_name = "".join(current_function_name)
            function_name = _re_comma_ending_parentheses.sub(")", function_name)
            function_name = _re_opening_parentheses_whitespace.sub("(", function_name)
            public_functions.append(function_name)
            # Reset
            trying_to_find = False
            bracket_found = False
            current_function_name: list[str] = []

    return public_functions

In [ ]:
public_functions: list[str] = []

for root, dirs, files in os.walk(_path1):
    for file in files:
        if file.endswith(".go"):
            full_path = os.path.join(root, file)
            l1 = _process_file(full_path)
            public_functions.extend(l1)
public_functions.sort()
print(len(public_functions))

In [ ]:
# output now
interface_lines: list[str] = []
interface_lines.append(f"type {_interface_name} interface" + " {")

for fn in public_functions:
    interface_lines.append("\t" + fn)

interface_lines.append("}")

outdata = "\n".join(interface_lines)
print(outdata)